# 티켓을 묶어보자! (임베딩 저장 ver)
https://gemini.google.com/app/1c8de67b438fd194?hl=ko

https://gemini.google.com/app/dca02e9096e76d3f?hl=ko

## Library Installation

In [1]:
!pip install pandas

In [3]:
!pip install --upgrade numexpr

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numexpr: filename=numexpr-2.14.1-cp310-cp310-linux_x86_64.whl size=163286 sha256=cb084c859af016b72ac24784141857eb1e41a9c51c091130238eddb234bde4df
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c4/16/c7/6252b16a24e3bb6f20a4e95105f016c31fffa5f38f64b46d5c
Successfully built numexpr
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.7.3
    Uninstalling numexpr-2.7.3:
      Successfully uninstalled numexpr-2.7.3


In [4]:
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 20.6 MB/s  0:00:15m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 39.6 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 186.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 92.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 21.2 MB/s  0:00:13m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 67.2 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 62.0 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 97.5 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 65.1 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 52.6 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Retrieve Data

In [72]:
import pandas as pd
import ast
import os
from IPython.display import display

In [73]:
# 추론(Inference) 단계와의 호환성을 위해 남겨두는 필드 목록
fields_to_include = [
    "CmdLine", "DetectSubType", "DetectTime",
    "EventSubType", "EventTime", "EventType", "FileName", "FileType", "HostName",
    "IP", "IsKnown", "Platform", "ProcName", "ProcPath",
    "RuleName", "SHA256", "Tactic", "TacticID", "Technique", "TechniqueID",
    "threat_label.verdict", "threat_label.scenario", "threat_label.case_id",
    "SuspliciousInfo", "ResponseInfo"
]

# 1. 증폭된 CSV 데이터 파일 로드 
csv_files = [
    "./data/augmented_training_data(account_manipulation).csv"
]

df_list = []
for file in csv_files:
    if os.path.exists(file):
        print(f"Loading '{file}'...")
        try:
            # 인코딩 문제 발생 시 'cp949'나 'euc-kr'로 변경 시도
            temp_df = pd.read_csv(file, encoding='utf-8-sig')
            df_list.append(temp_df)
        except Exception as e:
            print(f"Error loading {file}: {e}")
    else:
        print(f"Warning: '{file}' 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

if df_list:
    df = pd.concat(df_list, ignore_index=True)
    print(f"\n총 {len(df)}개의 학습 데이터를 로드했습니다.")
else:
    # 파일이 없을 경우 빈 프레임 생성 (에러 방지) 또는 예외 발생
    raise FileNotFoundError("로딩할 데이터 파일이 없습니다. 파일명을 확인해주세요.")

# 2. [핵심] 문자열로 깨진 리스트 객체 복원
# CSV에는 리스트가 "['A', 'B']" 같은 문자열로 저장됨 이를 실제 파이썬 리스트 ['A', 'B']로 변환
list_columns_to_fix = [
    'ResponseInfo_detect_terminateprocess', 
    'Tactic',
    'TacticID',
    'Technique',
    'TechniqueID'
]

print("리스트 컬럼 복원 중...")
for col in list_columns_to_fix:
    if col in df.columns:
        # 값이 문자열이고 '['로 시작하는 경우에만 literal_eval 수행
        # 데이터에 NaN이 있을 수 있으므로 안전하게 처리
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith('[') else [])

print("데이터 준비 완료!")
print("\n데이터프레임 샘플:")
display(df.head())

Loading './data/augmented_training_data(account_manipulation).csv'...

총 2000개의 학습 데이터를 로드했습니다.
리스트 컬럼 복원 중...
데이터 준비 완료!

데이터프레임 샘플:


,DetectSubType,EventType,ProcName,FileName,IP,CmdLine,DetectTime,EventSubType,EventTime,ProcPath,HostName,RuleName,ResponseInfo_detect_terminateprocess,threat_label_scenario,threat_label_verdict,threat_label_case_id,Tactic,TacticID,TechniqueID,Technique
0,Anomaly,process,cmd.exe,net.exe,192.168.3.172,"cmd.exe /C net user /add ""toor_8962"" ""toor_896...",1763087281022,ProcessStart,1763087280924,C:\Windows\system32\cmd.exe,DEV-C1MUS20,사용자 계정 등록 시도,"[{'Description': '자식 프로세스', 'ProcName': 'net.e...",Account Manipulation - Create new Windows admi...,malicious,AUG-CASE-E753DCF5,[Persistence],[TA0003],[T1136],[Create Account]
1,Anomaly,process,cmd.exe,net.exe,192.168.3.172,"cmd.exe /C net user /add ""toor_8962"" ""toor_896...",1763087280837,ProcessStart,1763087280764,C:\Windows\system32\cmd.exe,DEV-C1MUS20,사용자 계정 등록 시도,"[{'Description': '자식 프로세스', 'ProcName': 'net.e...",Account Manipulation - Create new Windows admi...,malicious,AUG-CASE-E753DCF5,[Persistence],[TA0003],[T1136],[Create Account]
2,Anomaly,process,cmd.exe,net.exe,192.168.133.123,"cmd.exe /C net user /add ""support_lzo"" ""suppor...",1762435321002,ProcessStart,1762435320924,C:\Windows\system32\cmd.exe,DEV-QNW7M13,사용자 계정 등록 시도,"[{'Description': '자식 프로세스', 'ProcName': 'net.e...",Account Manipulation - Create new Windows admi...,malicious,AUG-CASE-8F9B2E10,[Persistence],[TA0003],[T1136],[Create Account]
3,Anomaly,process,cmd.exe,net.exe,192.168.133.123,"cmd.exe /C net user /add ""support_lzo"" ""suppor...",1762435320851,ProcessStart,1762435320764,C:\Windows\system32\cmd.exe,DEV-QNW7M13,사용자 계정 등록 시도,"[{'Description': '자식 프로세스', 'ProcName': 'net.e...",Account Manipulation - Create new Windows admi...,malicious,AUG-CASE-8F9B2E10,[Persistence],[TA0003],[T1136],[Create Account]
4,Anomaly,process,cmd.exe,net.exe,192.168.250.41,"cmd.exe /C net user /add ""aws_cli_3021"" ""aws_c...",1761729481020,ProcessStart,1761729480924,C:\Windows\system32\cmd.exe,LAPTOP-XO1KROA,사용자 계정 등록 시도,"[{'Description': '자식 프로세스', 'ProcName': 'net.e...",Account Manipulation - Create new Windows admi...,malicious,AUG-CASE-6F418F02,[Persistence],[TA0003],[T1136],[Create Account]


## 기본 feature 추출
- 범주형 feature: EventType, EventSubType, RuleName, DetectSubType
- 텍스트 feature: CmdLine, ProcPath, FileName
- 시간 feature: EventTime

In [74]:
import torch
import torch.nn as nn
import re
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [75]:
# feature engineering을 위한 data frame 복사
feature_df = df.copy()

# 각 feature 처리 단계에서 생성된 결과를 저장할 리스트
# feature_list = [feature_df]
feature_list = []

In [76]:
columns_to_check = ['EventType', 'EventSubType', 'DetectSubType', 'RuleName', 'RuleId']

for col in columns_to_check:
    if col in df.columns:
        print(f"\n[{col}] unique values:")
        print(df[col].unique())
    else:
        print(f"\n[{col}] column not found in DataFrame")


[EventType] unique values:
['process']

[EventSubType] unique values:
['ProcessStart']

[DetectSubType] unique values:
['Anomaly']

[RuleName] unique values:
['사용자 계정 등록 시도']

[RuleId] column not found in DataFrame


### Temporal Features
- EventTime 기준으로 Time Difference 계산해 새로운 컬럼 만들기
- 공격의 연속성 파악
- 공격의 인과 관계 순서를 파악 

In [77]:
print("### 1. 시간 피처 처리 시작 ###")

# HostName 또는 EventTime 컬럼이 없는 경우를 대비한 예외 처리
if 'HostName' in feature_df.columns and 'EventTime' in feature_df.columns:
    feature_df['EventTime_dt'] = pd.to_datetime(feature_df['EventTime'], unit='ms')
    # 데이터를 정렬하고 인덱스를 리셋하여 순서를 고정
    feature_df = feature_df.sort_values(by=['HostName', 'EventTime_dt']).reset_index(drop=True)
    
    time_diff = feature_df.groupby('HostName')['EventTime_dt'].diff().dt.total_seconds().fillna(0)
    feature_df['time_diff_sec'] = np.maximum(time_diff, 0)
    
    # 생성된 피처를 feature_list에 추가
    feature_list.append(feature_df[['time_diff_sec']])
    print("-> 완료: 시간 차이(time_diff_sec) 피처 생성.\n")
else:
    print("-> 경고: 'HostName' 또는 'EventTime' 컬럼이 없어 시간 피처를 생성하지 않았습니다.\n")

### 1. 시간 피처 처리 시작 ###
-> 완료: 시간 차이(time_diff_sec) 피처 생성.



### Categorical Features
EventType, EventSubType, RuleName, DetectSubType -> 원-핫 인코딩

In [78]:
print("### 2. 범주형 피처 처리 시작 ###")

# 처리할 범주형 컬럼 목록
categorical_cols = ['EventType', 'EventSubType', 'DetectSubType', 'RuleName']

# 결측값(NaN)을 'Unknown' 문자열로 대체
# for col in categorical_cols:
#     feature_df[col] = feature_df[col].fillna('Unknown')
for col in categorical_cols:
    if col in feature_df.columns:
        feature_df[col] = feature_df[col].fillna('Unknown')

# Pandas의 get_dummies를 사용한 원-핫 인코딩
# 각 카테고리 값을 새로운 컬럼으로 만들고, 해당하면 1 아니면 0으로 채움
categorical_features = pd.get_dummies(feature_df[categorical_cols], prefix=categorical_cols)
feature_list.append(categorical_features)

print(f"원-핫 인코딩으로 {categorical_features.shape[1]}개의 피처 생성 완료.")
print("생성된 피처 예시 (상위 5개):")
print(categorical_features.head())
print("-" * 50)

### 2. 범주형 피처 처리 시작 ###
원-핫 인코딩으로 4개의 피처 생성 완료.
생성된 피처 예시 (상위 5개):
   EventType_process  EventSubType_ProcessStart  DetectSubType_Anomaly  \
0               True                       True                   True   
1               True                       True                   True   
2               True                       True                   True   
3               True                       True                   True   
4               True                       True                   True   

   RuleName_사용자 계정 등록 시도  
0                   True  
1                   True  
2                   True  
3                   True  
4                   True  
--------------------------------------------------


In [79]:
# 전체 컬럼 목록을 csv 파일로 저장
output_file = 'categorical_features.csv'
categorical_features.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"전체 피처가 '{output_file}' 파일에 저장되었습니다.")

전체 피처가 'categorical_features.csv' 파일에 저장되었습니다.


### Text Features
`만들어진 cmdline_features_df에서 랜섬이라는 단어가 들어간 column과 특정 이름이 들어간 column drop해야 할 듯`
- CmdLine -> TF-IDF Vectorizer
- ProcName, FileName은 어떻게 하지?
- TfidVectorizer: 단순히 단어의 빈도수만 세는 것이 아니라, 모든 문서에서 공통적으로 많이 나타나는 단어에는 낮은 가중치를, 특정 문서에만 집중적으로 나타나는 단어에는 높은 가중치 부여

#### CmdLine

In [80]:
print("### 3. 텍스트 피처 처리 시작 ###")

if 'CmdLine' in feature_df.columns:
    feature_df['CmdLine'] = feature_df['CmdLine'].fillna('')
    
    tfidf_vectorizer = TfidfVectorizer(
        max_features=3000, # 랜덤 변수를 잡기 위해 충분히 크게 생성(공통 단어뿐만 아니라 희귀한 랜덤 문자열도 피처로 포함하기 위해
        ngram_range=(1, 2),
        min_df=1 # 단 한 번만 등장하는 단어도 무시하지 않음
    )
    cmdline_features_matrix = tfidf_vectorizer.fit_transform(feature_df['CmdLine'])
    cmdline_features_df = pd.DataFrame(
        cmdline_features_matrix.toarray(),
        columns=[f"cmd_{name}" for name in tfidf_vectorizer.get_feature_names_out()]
    )
    feature_list.append(cmdline_features_df)
    print(f"-> 완료: TF-IDF로 {cmdline_features_df.shape[1]}개 피처 생성.\n")
else:
    print("-> 경고: 'CmdLine' 컬럼이 없어 텍스트 피처를 생성하지 않았습니다.\n")

### 3. 텍스트 피처 처리 시작 ###
-> 완료: TF-IDF로 3000개 피처 생성.



In [81]:
# 전체 컬럼 목록을 csv 파일로 저장
output_file = 'cmdline_features_df.csv'
cmdline_features_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"전체 피처가 '{output_file}' 파일에 저장되었습니다.")

전체 피처가 'cmdline_features_df.csv' 파일에 저장되었습니다.


### Path Features
- ProcPath -> Tokenizer + LSTM

#### ProcPath

기본 설정 및 경로 토큰화 함수 정의

In [82]:
print("### 4. 경로(ProcPath, FileName) 피처 처리 시작 ###")

def tokenize_path(path):
    """
    경로 문자열을 입력받아 토큰 리스트로 변환하는 함수.
    - None이나 NaN 값은 빈 리스트로 처리.
    - 모든 문자를 소문자로 변환.
    - '\', '/', '.' 를 기준으로 분리.
    """
    if not isinstance(path, str):
        return []
    # 정규표현식을 사용하여 여러 구분자로 분리
    tokens = re.split(r'[\\/.]', path.lower())
    # 빈 문자열 제거 (예: '/usr/bin' -> ['', 'usr', 'bin'])
    return [token for token in tokens if token]

### 4. 경로(ProcPath, FileName) 피처 처리 시작 ###


PyTorch Sequential Model 정의

In [83]:
class PathEmbedder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PathEmbedder, self).__init__()
        
        # 1. 임베딩 레이어: 정수 인덱스를 밀집 벡터로 변환
        # padding_idx=0: <pad> 토큰은 학습 과정에서 무시하도록 설정
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. LSTM 레이어: 임베딩된 벡터 시퀀스를 입력받아 문맥을 학습
        # batch_first=True: 입력 텐서의 차원을 (batch_size, sequence_length, ...) 순으로 받음
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
    def forward(self, x):
        # paths_tensor: (batch_size, max_sequence_length)
        
        # 1. 임베딩
        embedded = self.embedding(x)
        # embedded: (batch_size, max_sequence_length, embedding_dim)
        
        # 2. LSTM
        # LSTM의 마지막 hidden state가 경로 전체의 문맥 정보를 압축한 결과물
        _, (hidden, _) = self.lstm(embedded)
        # hidden: (1, batch_size, hidden_dim)
        
        # 최종 출력 벡터의 차원을 (batch_size, hidden_dim)으로 조정
        final_embedding = hidden.squeeze(0)
        
        return final_embedding

임베딩 추출 실행

In [84]:
def paths_to_tensor(paths, word_to_idx):
    """
    경로 문자열 리스트를 토큰화, 인덱싱, 패딩하여 텐서로 변환하는 함수.
    """

    # 1. 토큰화 및 정수 인덱싱
    # 사전에 없으면 <unk> 토큰의 인덱스로, 있으면 해당 토큰의 인덱스로 변환
    indexed_paths = [[word_to_idx.get(token, word_to_idx['<unk>']) for token in tokenize_path(p)] for p in paths]

    # 2. 패딩 (길이 맞추기)
    max_len = max(len(p) for p in indexed_paths) if indexed_paths else 0
    padded_paths = [p + [word_to_idx['<pad>']] * (max_len - len(p)) for p in indexed_paths]
    
    # 3. PyTorch 텐서로 변환
    return torch.LongTensor(padded_paths)

전체 경로 토큰에 대한 단어 사전 구축

In [85]:
# ProcPath 컬럼이 있는지 먼저 확인
if 'ProcPath' in feature_df.columns:
    # 단어 사전(Vocabulary)을 ProcPath 기준으로 구축
    all_path_tokens = [token for path in feature_df['ProcPath'].dropna() for token in tokenize_path(path)]
    vocab = ['<pad>', '<unk>'] + [token for token, _ in Counter(all_path_tokens).most_common(2000)]
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    vocab_size = len(word_to_idx)

    # 모델 인스턴스 생성
    EMBEDDING_DIM = 32  # 각 토큰을 표현할 벡터의 차원
    HIDDEN_DIM = 64    # 경로 전체를 표현할 최종 임베딩 벡터의 차원
    path_embedder_model = PathEmbedder(vocab_size, EMBEDDING_DIM, HIDDEN_DIM)
    path_embedder_model.eval()

    # ProcPath에 대한 임베딩 추출
    with torch.no_grad():
        procpath_tensor = paths_to_tensor(feature_df['ProcPath'].tolist(), word_to_idx)
        procpath_embeddings = path_embedder_model(procpath_tensor).numpy()
        procpath_df = pd.DataFrame(procpath_embeddings, columns=[f'pp_emb_{i}' for i in range(HIDDEN_DIM)])
        feature_list.append(procpath_df)
        print(f"-> 완료: ProcPath에 대한 {HIDDEN_DIM}차원 임베딩 피처 생성.\n")
else:
    print("-> 경고: 'ProcPath' 컬럼이 없어 경로 피처를 생성하지 않았습니다.\n")

-> 완료: ProcPath에 대한 64차원 임베딩 피처 생성.



### MITRE ATT&CK Features

In [86]:
from sklearn.preprocessing import MultiLabelBinarizer

In [87]:
print("### 5. MITRE ATT&CK 정보('Tactic', 'TacticID', 'Technique', 'TechniqueID') 피처 처리 시작 ###")

# Tactic, Technique의 'ID' 컬럼만 선택하여 처리
attack_cols_by_id = ['TacticID', 'TechniqueID'] 
all_attack_features = []

attack_mlbs = {}
for col in attack_cols_by_id:
    if col in feature_df.columns:
        # 결측치(NaN)를 빈 리스트로 변환
        series = feature_df[col].apply(lambda d: d if isinstance(d, list) else [])
        
        # MultiLabelBinarizer 초기화 및 학습/변환
        mlb = MultiLabelBinarizer()
        encoded_matrix = mlb.fit_transform(series)
        attack_mlbs[col] = mlb
        
        # 인코딩된 결과가 있을 경우에만 데이터프레임으로 만들고 리스트에 추가
        if encoded_matrix.shape[1] > 0:
            # 컬럼 이름을 'TacticID_TA0001'과 같이 생성
            attack_df = pd.DataFrame(encoded_matrix, columns=[f"{col}_{cls}" for cls in mlb.classes_])
            all_attack_features.append(attack_df)
        else:
            print(f"   - '{col}' 컬럼에 유효한 값이 없어 피처를 생성하지 않습니다.")

if all_attack_features:
    # 생성된 모든 ATT&CK 피처 데이터프레임을 하나로 합침
    attack_features_combined = pd.concat(all_attack_features, axis=1)
    feature_list.append(attack_features_combined)
    print(f"-> 완료: ATT&CK 관련 피처 {attack_features_combined.shape[1]}개 생성.\n")
else:
    print("-> 완료: 처리할 ATT&CK 관련 컬럼이 없습니다.\n")

### 5. MITRE ATT&CK 정보('Tactic', 'TacticID', 'Technique', 'TechniqueID') 피처 처리 시작 ###
-> 완료: ATT&CK 관련 피처 2개 생성.



### ResponseInfo_detect_terminateprocess column
[proc_relation_type] <br>
부모/자식 정보가 모두 있으면 'ParentChild'<br>
부모 없이 행위자만 있는 경우 'Self' <br>
정보가 없으면 'None' <br>
부모-자식 생성 관계인지, 아니면 단일 프로세스의 독립적인 악성 행위인지를 명확히 구분하여 모델에 알려줌 

[parent_is_system, child_is_system] <br>
시스템 프로세스 여부 <br>
추출 방법: SystemService 필드(true/false)를 그대로 사용 <br>
기대 효과: explorer.exe나 svchost.exe 같은 정상 시스템 프로세스가 악성 행위의 부모가 되는 경우가 많음. 이 플래그는 "정상 프로세스를 이용한 공격" 패턴을 학습하는 데 결정적인 역할을 함.<br>

[is_parent_child_path_similar]<br>
경로 유사성 <br>
추출 방법: 부모의 ProcPath 디렉터리와 자식의 ProcPath 디렉터리가 동일한지 여부를 확인. (예: 둘 다 C:\Users\...\AppData\Local\Temp에 위치)<br>
기대 효과: 특정 폴더 내에서 파일 생성, 실행, 삭제가 연쇄적으로 일어나는 공격(예: Dropper)의 특징을 잡아낼 수 있음.<br>

[parent_child_pair]<br>
부모-자식 쌍(Pair) 피처화<br>
추출 방법: ParentProcName과 ChildProcName을 -> 기호로 연결한 새로운 문자열 피처 많듦. (예: 'explorer.exe->malware.exe')<br>

기대 효과: explorer.exe라는 단일 정보보다 'explorer.exe->malware.exe'라는 관계 정보가 훨씬 더 강력한 악성 지표가 되고 이 새로운 범주형 피처를 원-핫 인코딩하면 모델이 특정 생성 패턴을 직접적으로 학습 가능<br>

[이벤트 프로세스와의 관계 정의]
핵심: ResponseInfo의 정보와 이벤트 최상위 레벨의 ProcName을 비교합니다.

추출 방법:

ResponseInfo의 자식 프로세스 이름이 최상위 ProcName과 같다면, 이 이벤트는 '생성된(Spawned)' 이벤트입니다.

ResponseInfo에 부모 프로세스만 있고, 그 이름이 최상위 ProcName과 같다면, 이 이벤트는 '생성하는(Spawning)' 이벤트입니다.

기대 효과: 이벤트의 역할을 명확하게 정의하여 시간의 흐름에 따른 인과관계를 모델이 더 잘 이해하도록 돕습니다.



---
'기원' vs '수행' == '부모' vs '자식/행위자'
- 의심 행위를 수행한 프로세스 -> 자식/행위자 역할
- 커맨드/스크립트를 실행한 프로세스의 부모 프로세스 -> 부모
- 부모 프로세스 -> 부모
- 자식 프로세스 -> 자식/행위자 역할
- 커맨드/스크립트를 실행한 프로세스 -> 자식/행위자 역할


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [89]:
print("### 6. ResponseInfo_detect_terminateprocess 정보 피처 처리 시작 ###")

# ResponseInfo_detect_terminateprocess 필드에서 키워드 기반으로 피처 추
def extract_advanced_process_info(row):

    # 기본값 설정
    parent_info = {'ProcName': 'Unknown', 'SystemService': False, 'ProcPath': '', 'CmdLine': ''}
    child_info = {'ProcName': 'Unknown', 'SystemService': False, 'ProcPath': '', 'CmdLine': ''}

    # 정보 추출
    if isinstance(row['ResponseInfo_detect_terminateprocess'], list):
        for item in row['ResponseInfo_detect_terminateprocess']:
            desc = item.get('Description', '')

            if '부모' in desc: # '부모' 키워드가 포함되어 있다면 parent_info로 간주
                parent_info = item
            elif any(keyword in desc for keyword in ['자식', '의심 행위', '커맨드/스크립트']): # 관련 키워드가 있으면 child_info로 간주
                child_info = item
    # 1. 관계 유형 피처
    if parent_info['ProcName'] != 'Unknown' and child_info['ProcName'] != 'Unknown':
        relation_type = 'ParentChild'
    elif child_info['ProcName'] != 'Unknown':
        relation_type = 'Self' # 부모 없이 행위자만 있는 경우
    else:
        relation_type = 'None'

    # 2. 시스템 프로세스 여부
    parent_is_system = parent_info.get('SystemService', False)
    child_is_system = child_info.get('SystemService', False)

    # 3. 경로 유사성
    try:
        parent_dir = '\\'.join(parent_info.get('ProcPath', '').split('\\')[:-1])
        child_dir = '\\'.join(child_info.get('ProcPath', '').split('\\')[:-1])
        is_path_similar = (parent_dir == child_dir) and parent_dir != ''
    except:
        is_path_similar = False

    # 4. 부모-자식 쌍
    parent_child_pair = f"{parent_info.get('ProcName', 'Unknown')}->{child_info.get('ProcName', 'Unknown')}"

    return pd.Series([
        relation_type,
        parent_is_system,
        child_is_system,
        is_path_similar,
        parent_child_pair,
        parent_info.get('CmdLine', ''), 
        child_info.get('CmdLine', '')
    ])

if 'ResponseInfo_detect_terminateprocess' in feature_df.columns:
    # 데이터프레임의 각 행에 함수 적용
    new_proc_features = df.apply(extract_advanced_process_info, axis=1)
    new_proc_features.columns = [
        'proc_relation_type', 'parent_is_system', 'child_is_system',
        'is_path_similar', 'parent_child_pair', 'parent_cmdline', 'child_cmdline'
    ]

    # 범주형/불리언 피처는 원-핫 인코딩 또는 그대로 사용
    proc_categorical_features = pd.get_dummies(new_proc_features[[
        'proc_relation_type', 'parent_is_system', 'child_is_system', 
        'is_path_similar', 'parent_child_pair'
    ]])
    feature_list.append(proc_categorical_features)
    print(f"-> 완료: 프로세스 관계 피처 {proc_categorical_features.shape[1]}개 생성.")

    # 부모/자식 CmdLine은 TF-IDF로 벡터화
    # tfidf_parent_cmd = TfidfVectorizer(max_features=30, prefix='pcmd_')
    tfidf_parent_cmd = TfidfVectorizer(max_features=30)
    parent_cmd_vectors = tfidf_parent_cmd.fit_transform(new_proc_features['parent_cmdline'].fillna(''))
    
    parent_cmd_df = pd.DataFrame(
        parent_cmd_vectors.toarray(),
        columns=[f"pcmd_{col}" for col in tfidf_parent_cmd.get_feature_names_out()]  # ← 접두어 직접 추가
    )
    feature_list.append(parent_cmd_df)

    # 자식 CmdLine TF-IDF
    # tfidf_child_cmd = TfidfVectorizer(max_features=30, prefix='ccmd_')
    tfidf_child_cmd = TfidfVectorizer(max_features=30)
    child_cmd_vectors = tfidf_child_cmd.fit_transform(new_proc_features['child_cmdline'].fillna(''))
    
    # child_cmd_df = pd.DataFrame(child_cmd_vectors.toarray(), columns=tfidf_child_cmd.get_feature_names_out())
        
    child_cmd_df = pd.DataFrame(
        child_cmd_vectors.toarray(),
        columns=[f"ccmd_{col}" for col in tfidf_child_cmd.get_feature_names_out()]  # ← 접두어 직접 추가
    )
    feature_list.append(child_cmd_df)

    print(f"-> 완료: 부모/자식 CmdLine 피처 {parent_cmd_df.shape[1] + child_cmd_df.shape[1]}개 생성.\n")
else:
    print("-> 경고: 'ResponseInfo_detect_terminateprocess' 컬럼이 없습니다.\n")


### 6. ResponseInfo_detect_terminateprocess 정보 피처 처리 시작 ###
-> 완료: 프로세스 관계 피처 5개 생성.
-> 완료: 부모/자식 CmdLine 피처 60개 생성.



### 최종 모든 feature dataframe 병합 및 저장

In [90]:
import pandas as pd
from IPython.display import display

print("### 7. 최종 피처 병합 ###")

# feature_list에 저장된 모든 피처 데이터프레임을 수평으로(axis=1) 합칩니다.
# 모든 피처가 feature_df와 동일한 인덱스(0, 1, 2...)를 기준으로 정렬되어 있습니다.
try:
    final_features_df = pd.concat(feature_list, axis=1)
    print(f"-> 완료: 모든 개별 피처 DataFrames 병합. (총 {final_features_df.shape[1]}개 피처)")

    # 모델 학습에 필요한 식별자, 레이블과 피처를 최종적으로 결합합니다.
    # (중요) 1단계에서 정렬/인덱스 리셋을 마친 feature_df에서 컬럼을 가져옵니다.
    key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
    
    # feature_df에 실제 존재하는 키 컬럼만 선택합니다.
    existing_key_columns = [col for col in key_columns if col in feature_df.columns]
    
    # 정렬된 키 컬럼 + 피처 컬럼을 수평으로 결합
    final_model_input_df = pd.concat([
        feature_df[existing_key_columns].reset_index(drop=True), # 안전을 위해 인덱스 리셋
        final_features_df.reset_index(drop=True)
    ], axis=1)

    print(f"-> 완료: 최종 모델 입력 데이터프레임 생성. (shape: {final_model_input_df.shape})")
    print("\n최종 데이터프레임 샘플 (상위 5개):")
    display(final_model_input_df.head())
    
    # 다음 단계(모델 학습)를 위해 파일로 저장
    output_filename = "final_model_input.csv"
    final_model_input_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"\n✅ 모든 피처 엔지니어링 완료! '{output_filename}' 파일로 저장되었습니다.")

except ValueError as e:
    print(f"\n[오류] 피처 병합 중 문제가 발생했습니다: {e}")
    print("feature_list에 있는 DataFrame들의 행(row) 개수가 일치하는지 확인해주세요.")

### 7. 최종 피처 병합 ###
-> 완료: 모든 개별 피처 DataFrames 병합. (총 3136개 피처)
-> 완료: 최종 모델 입력 데이터프레임 생성. (shape: (2000, 3141))

최종 데이터프레임 샘플 (상위 5개):


,HostName,EventTime,threat_label_case_id,threat_label_verdict,threat_label_scenario,time_diff_sec,EventType_process,EventSubType_ProcessStart,DetectSubType_Anomaly,RuleName_사용자 계정 등록 시도,...,ccmd_user_1549,ccmd_user_1697,ccmd_user_2543,ccmd_user_2712,ccmd_user_3360,ccmd_user_3547,ccmd_user_3717,ccmd_user_3978,ccmd_user_4817,ccmd_user_4955
0,DESKTOP-04E4KCU,1763831820782,AUG-CASE-A0B0D497,malicious,Account Manipulation - Create new Windows admi...,0.00,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DESKTOP-04E4KCU,1763831820942,AUG-CASE-A0B0D497,malicious,Account Manipulation - Create new Windows admi...,0.16,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DESKTOP-0CMA1DH,1762778760818,AUG-CASE-0DFA70B1,malicious,Account Manipulation - Create new Windows admi...,0.00,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DESKTOP-0CMA1DH,1762778760978,AUG-CASE-0DFA70B1,malicious,Account Manipulation - Create new Windows admi...,0.16,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DESKTOP-0S4N8GG,1763484360832,AUG-CASE-0BFEF55D,malicious,Account Manipulation - Create new Windows admi...,0.00,True,True,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



✅ 모든 피처 엔지니어링 완료! 'final_model_input.csv' 파일로 저장되었습니다.


📈 다음 단계: 시퀀스 생성 및 LSTM 모델 학습
이제부터는 본격적으로 AI 모델을 구축하고 학습하는 과정입니다. 전체 과정은 아래와 같이 진행됩니다.

데이터셋 재구성: 이벤트 벡터들을 시간 순서대로 묶어 '시퀀스(Sequence)' 데이터셋을 생성합니다.

샴 네트워크 모델 구축: 두 개의 시퀀스를 입력받아 유사도를 출력하는 LSTM 기반의 샴 네트워크를 정의합니다.

모델 학습: '같은 공격'에서 나온 시퀀스 쌍은 가깝게, '다른 공격'에서 나온 시퀀스 쌍은 멀게 만드는 방향으로 모델을 학습시킵니다.

아래는 이 모든 과정을 담은 PyTorch 코드입니다.

⚠️ 중요: 이 코드는 PyTorch 라이브러리가 필요하며, 현재 실행 환경에서는 구동되지 않습니다. 사용자님의 로컬 개발 환경(PC, Colab, 등)에 PyTorch를 설치하신 후 실행해주세요.

## MLP 기반 샴 네트워크

### traning

In [91]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
import random
from tqdm import tqdm
import os

#### 1. hyper parameterst 설정

In [101]:
# -- 1. 하이퍼파라미터 설정 --
INPUT_FILE = "final_model_input.csv"
MODEL_OUTPUT_DIR = "model_checkpoint/v2" # 모델 저장용 디렉터리
BEST_MODEL_NAME = "best_mlp_base_network.pth"

# 데이터 분리 비율
VAL_SPLIT_RATIO = 0.15
TEST_SPLIT_RATIO = 0.15

# 모델 파라미터
EMBEDDING_SIZE = 64  # 이벤트 벡터를 압축할 최종 차원 (64차원 '지문')
MARGIN = 2.0         # Contrastive Loss의 마진 값

# 훈련 파라미터
EPOCHS = 200           # 전체 데이터셋 반복 훈련 횟수
BATCH_SIZE = 64        # 한 번에 훈련시킬 페어(pair)의 개수
LEARNING_RATE = 0.001
PATIENCE = 15          # 15 Epoch 동안 성능 향상 없으면 조기 종료

#### 2. Pytorch 데이터셋 클래스 정의

In [93]:
class EventPairDataset(Dataset):
    """
    DataFrame을 직접 입력받아
    (이벤트 A, 이벤트 B, 레이블) 페어를 생성하는 클래스
    """
    def __init__(self, dataframe):
        # 1. 키 컬럼과 피처 컬럼 분리
        key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
        existing_key_columns = [col for col in dataframe.columns if col in key_columns]
        feature_cols = [col for col in dataframe.columns if col not in existing_key_columns]
        
        # 2. 피처 데이터와 레이블(case_id) 저장
        self.features = dataframe[feature_cols].values.astype(np.float32)
        self.labels = dataframe['threat_label_case_id'].values
        
        # 3. Positive Pair를 빠르게 찾기 위한 맵(dict) 생성
        self.label_to_indices = defaultdict(list)
        for idx, label in enumerate(self.labels):
            self.label_to_indices[label].append(idx)
            
        self.unique_labels = list(self.label_to_indices.keys())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        anchor_event = self.features[index]
        anchor_label = self.labels[index]
        
        if random.random() > 0.5:
            # Positive 페어 (Label = 1.0)
            positive_list = self.label_to_indices[anchor_label]
            if len(positive_list) == 1:
                positive_index = index
            else:
                # 자기 자신을 제외하고 랜덤 선택 (만약 자기 자신만 남으면 그냥 자기 자신)
                positive_index = random.choice([i for i in positive_list if i != index] or [index])
                
            pair_event = self.features[positive_index]
            label = 1.0
        else:
            # Negative 페어 (Label = 0.0)
            # 다른 case_id에서 랜덤하게 이벤트 선택
            negative_label = random.choice([l for l in self.unique_labels if l != anchor_label])
            negative_index = random.choice(self.label_to_indices[negative_label])
            
            pair_event = self.features[negative_index]
            label = 0.0
            
        return (
            torch.tensor(anchor_event, dtype=torch.float32),
            torch.tensor(pair_event, dtype=torch.float32),
            torch.tensor([label], dtype=torch.float32)
        )

#### 3. AI 모델 및 손실 함수 정의

In [114]:
class ResidualBlock(nn.Module):
    """
    잔차 연결(Residual Connection)이 포함된 블록
    Input -> [Linear -> BN -> ReLU -> Dropout -> Linear -> BN] + Input -> ReLU
    """
    def __init__(self, hidden_size, dropout=0.3):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size)
        )
        self.activation = nn.ReLU()

    def forward(self, x):
        # 잔차 연결: 입력(x)를 출력에 더해줌으로써 그래디언트 소실 방지
        return self.activation(x + self.block(x))

class AdvancedMlpNetwork(nn.Module):
    """
    ResNet 스타일의 개선된 임베딩 네트워크
    """
    def __init__(self, input_size, embedding_size, hidden_size=256, num_res_blocks=2):
        super(AdvancedMlpNetwork, self).__init__()
        
        # 1. 입력 투영 (Input Projection)
        self.input_layer = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU()
        )
        
        # 2. 잔차 블록 쌓기 (Deeper Network)
        self.res_blocks = nn.ModuleList([
            ResidualBlock(hidden_size) for _ in range(num_res_blocks)
        ])
        
        # 3. 최종 임베딩 출력
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, embedding_size) # 최종 임베딩
        )

    def forward(self, x):
        out = self.input_layer(x)
        for block in self.res_blocks:
            out = block(out)
        out = self.output_layer(out)
        return out

class SiameseNetwork(nn.Module):
    """
    두 개의 이벤트를 입력받는 샴 네트워크
    """
    def __init__(self, base_network):
        super(SiameseNetwork, self).__init__()
        self.base_network = base_network

    def forward(self, event1, event2):
        embedding1 = self.base_network(event1)
        embedding2 = self.base_network(event2)
        return embedding1, embedding2

# class ContrastiveLoss(nn.Module):
#     """
#     대조 손실 함수
#     Label=1 이면 거리를 가깝게, Label=0 이면 거리를 margin보다 멀게 학습
#     """
#     def __init__(self, margin=2.0):
#         super(ContrastiveLoss, self).__init__()
#         self.margin = margin
#         self.eps = 1e-9 # 0으로 나누는 것을 방지

#     def forward(self, output1, output2, label):
#         # 유클리드 거리 계산
#         euclidean_distance = nn.functional.pairwise_distance(output1, output2) + self.eps
        
#         # Contrastive Loss 계산
#         loss_contrastive = torch.mean(
#             (label) * torch.pow(euclidean_distance, 2) +
#             (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
#         )
#         return loss_contrastive
'''
유클리드 거리(Euclidean Distance)는 피처의 값 차이가 크면 거리가 확 벌어지기 때문에 지금 같은 상황에 취약\
대신 각도(Angle)를 보는 "코사인 유사도" 기반의 손실 함수로 바꾸면, 프로세스 종류가 달라도 "같은 계정명을 공유한다"는 방향성을 더 잘 잡아낼 수 있음
'''
# 유클리드 거리 대신 코사인 유사도 기반 손실 함수 사용 ->
class CosineContrastiveLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(CosineContrastiveLoss, self).__init__()
        self.margin = margin
        self.loss_fn = nn.CosineEmbeddingLoss(margin=margin)

    def forward(self, output1, output2, label):
        target = label.flatten().float()
        
        # 2. 0(다름)을 -1로 변환 (CosineEmbeddingLoss 요구사항: 1=유사, -1=다름)
        # 원본 label을 건드리지 않기 위해 clone() 사용 (안전장치)
        target = target.clone()
        target[target == 0] = -1
        return self.loss_fn(output1, output2, target)

In [95]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0, path='checkpoint.pth', verbose=False):
        self.patience = patience
        self.min_delta = min_delta
        self.path = path
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).  Saving model ...')
        # 샴 네트워크 전체가 아니라, 핵심인 Base Network의 가중치만 저장
        torch.save(model.base_network.state_dict(), self.path)

#### 4. 검증(Validation) 함수 정의

In [115]:
# def validate(model, loader, criterion, device):
#     """
#     검증 데이터셋으로 모델의 손실(loss)을 평가하는 함수
#     """
#     model.eval() # 평가 모드 (Dropout, BatchNorm 비활성화)
#     total_loss = 0
#     with torch.no_grad(): # 기울기 계산 비활성화 (메모리 절약, 속도 향상)
#         for anchor_event, pair_event, label in loader:
#             anchor_event = anchor_event.to(device)
#             pair_event = pair_event.to(device)
#             label = label.to(device)
            
#             output1, output2 = model(anchor_event, pair_event)
#             loss = criterion(output1, output2, label)
#             total_loss += loss.item()
            
#     return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for anchor_event, pair_event, label in loader:
            anchor_event = anchor_event.to(device)
            pair_event = pair_event.to(device)
            label = label.to(device)
            
            out1, out2 = model(anchor_event, pair_event)
            loss = criterion(out1, out2, label)
            total_loss += loss.item()
    return total_loss / len(loader)

#### 피처 엔지니어링 도구 저장 
-> 학습 데이터 기준으로 fit된 모든 전처리기의 최종 컬럼 목록을 preprocessors.joblib 파일로 저장

In [97]:
import joblib
import os

In [102]:
# print("### 8. (신규) 추론을 위한 피처 엔지니어링 도구 저장 ###")

# # 1. 저장할 디렉터리 생성 (모델 저장 위치와 동일하게 사용)
# MODEL_OUTPUT_DIR = "model_checkpoint/v2"
# os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# # 2. 전처리기 도구들을 딕셔너리에 수집
# # (주의: 변수명은 Sagemaker 노트북의 각 셀에서 사용된 최종 변수명과 일치해야 합니다)
# preprocessors = {}

# try:
#     # --- ES 조회 필드 목록 저장 ---
#     if 'fields_to_include' in locals():
#         preprocessors['fields_to_include'] = fields_to_include
#     else:
#         print("경고: 'fields_to_include' 변수를 찾을 수 없습니다.")

#     # --- 3. 텍스트 피처 (CmdLine) ---
#     if 'tfidf_vectorizer' in locals():
#         preprocessors['cmdline_tfidf'] = tfidf_vectorizer

#     # --- 4. 경로 피처 (ProcPath) ---
#     if 'word_to_idx' in locals():
#         preprocessors['path_word_to_idx'] = word_to_idx
#         preprocessors['path_embedder_params'] = {
#             'vocab_size': vocab_size,
#             'embedding_dim': EMBEDDING_DIM,
#             'hidden_dim': HIDDEN_DIM
#         }

#     # --- 5. ATT&CK 피처 (TacticID, TechniqueID) ---
#     if 'attack_mlbs' in locals():
#          preprocessors['attack_mlbs'] = attack_mlbs

#     # --- 6. ResponseInfo 피처 ---
#     if 'tfidf_parent_cmd' in locals():
#         preprocessors['parent_cmd_tfidf'] = tfidf_parent_cmd
#     if 'tfidf_child_cmd' in locals():
#         preprocessors['child_cmd_tfidf'] = tfidf_child_cmd
    
#     # --- 7. 최종 컬럼 목록 ---
#     if 'final_features_df' in locals():
#         preprocessors['final_feature_columns'] = final_features_df.columns.tolist()
#     else:
#         raise ValueError("final_features_df가 정의되지 않았습니다. 7단계가 정상 실행되었는지 확인하세요.")

#     # 3. 파일로 저장
#     preprocessor_path = os.path.join(MODEL_OUTPUT_DIR, "preprocessors.joblib")
#     joblib.dump(preprocessors, preprocessor_path)
    
#     print(f"✅ 전처리기 도구 저장 완료: {preprocessor_path}")
#     print(f"저장된 항목: {list(preprocessors.keys())}")

# except Exception as e:
#     print(f"[오류] 전처리기 저장 중 문제 발생: {e}")
#     print("-> 각 단계(3, 4, 5, 6, 7)의 변수명(tfidf_vectorizer, word_to_idx 등)이 올바른지 확인하세요.")



### 8. (신규) 추론을 위한 피처 엔지니어링 도구 저장 ###
✅ 전처리기 도구 저장 완료: model_checkpoint/v2/preprocessors.joblib
저장된 항목: ['fields_to_include', 'cmdline_tfidf', 'path_word_to_idx', 'path_embedder_params', 'attack_mlbs', 'parent_cmd_tfidf', 'child_cmd_tfidf', 'final_feature_columns']


In [ ]:
import joblib
import os

print("### 8. (수정됨) 추론을 위한 모든 피처 엔지니어링 도구 저장 ###")

# 1. 저장할 디렉터리 생성
MODEL_OUTPUT_DIR = "model_checkpoint/v2" 
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)

# 2. 전처리기 도구들을 딕셔너리에 수집
inference_artifacts = {}

try:
    # --- [1] 핵심 도구 (필수) ---
    # 1-1. 시간 스케일러 (이게 없으면 학습 데이터와 스케일이 안 맞음)
    if 'scaler' in locals():
        inference_artifacts['scaler'] = scaler
    else:
        print("⚠️ 경고: 'scaler' 변수가 없습니다. 시간 스케일링을 건너뛰었나요?")

    # 1-2. 최종 컬럼 목록 (입력 차원 맞추기용)
    if 'final_features_df' in locals():
        inference_artifacts['final_feature_columns'] = final_features_df.columns.tolist()
    else:
        raise ValueError("final_features_df가 없습니다. Step 7이 실행되었는지 확인하세요.")
        
    # 1-3. 모델 사이즈 정보
    inference_artifacts['model_input_size'] = INPUT_SIZE
    inference_artifacts['embedding_size'] = EMBEDDING_SIZE

    # --- [2] 텍스트 피처 (CmdLine) ---
    if 'tfidf_vectorizer' in locals():
        inference_artifacts['cmdline_tfidf'] = tfidf_vectorizer

    # --- [3] 경로 피처 (ProcPath) ---
    if 'word_to_idx' in locals() and 'path_embedder_model' in locals():
        inference_artifacts['path_word_to_idx'] = word_to_idx
        # [중요] 모델의 구조뿐만 아니라 '학습된 가중치(state_dict)'를 저장해야 함
        inference_artifacts['path_embedder_state'] = path_embedder_model.state_dict()
        inference_artifacts['path_embedder_params'] = {
            'vocab_size': vocab_size,
            'embedding_dim': EMBEDDING_DIM,
            'hidden_dim': HIDDEN_DIM
        }

    # --- [4] 기타 피처들 ---
    if 'attack_mlbs' in locals():
         inference_artifacts['attack_mlbs'] = attack_mlbs

    if 'tfidf_parent_cmd' in locals():
        inference_artifacts['parent_cmd_tfidf'] = tfidf_parent_cmd
        
    if 'tfidf_child_cmd' in locals():
        inference_artifacts['child_cmd_tfidf'] = tfidf_child_cmd
    
    # --- [5] (옵션) ES 조회 필드 목록 ---
    if 'fields_to_include' in locals():
        inference_artifacts['fields_to_include'] = fields_to_include

    # 3. 파일로 저장
    save_path = os.path.join(MODEL_OUTPUT_DIR, "full_inference_artifacts.joblib")
    joblib.dump(inference_artifacts, save_path)
    
    print(f"\n✅ 모든 추론 도구가 완벽하게 저장되었습니다: {save_path}")
    print(f"저장된 항목: {list(inference_artifacts.keys())}")

except Exception as e:
    print(f"\n[오류] 전처리기 저장 중 문제 발생: {e}")

#### 5. 훈련(Training) 스크립트 실행

In [104]:
if __name__ == "__main__":

    # -- [핵심] 그룹(case_id) 기반 데이터 분리 --
    print(f"'{INPUT_FILE}' 파일 로드 및 그룹 기반 분리 시작...")
    df = pd.read_csv(INPUT_FILE)

    # 1. 고유한 case_id 리스트 추출
    unique_case_ids = df['threat_label_case_id'].unique()
    np.random.shuffle(unique_case_ids) # case_id 리스트를 섞음

    # 2. Test, Validation, Train 용 case_id 분리
    test_size = int(len(unique_case_ids) * TEST_SPLIT_RATIO)
    val_size = int(len(unique_case_ids) * VAL_SPLIT_RATIO)

    test_ids = unique_case_ids[:test_size]
    val_ids = unique_case_ids[test_size : test_size + val_size]
    train_ids = unique_case_ids[test_size + val_size:]

    # 3. case_id 리스트를 기준으로 DataFrame 분리
    train_df = df[df['threat_label_case_id'].isin(train_ids)]
    val_df = df[df['threat_label_case_id'].isin(val_ids)]
    test_df = df[df['threat_label_case_id'].isin(test_ids)]

    print("데이터 분리 완료:")
    print(f"  - Train Set: {len(train_ids)}개 Case ID, {len(train_df)}개 이벤트")
    print(f"  - Val Set  : {len(val_ids)}개 Case ID, {len(val_df)}개 이벤트")
    print(f"  - Test Set : {len(test_ids)}개 Case ID, {len(test_df)}개 이벤트")

    # 4. 데이터셋 및 데이터로더 준비
    train_dataset = EventPairDataset(dataframe=train_df)
    val_dataset = EventPairDataset(dataframe=val_df)
    # Test 데이터셋은 나중에 최종 평가 시 사용

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # 5. 모델, 손실함수, 옵티마이저 초기화
    INPUT_SIZE = train_dataset.features.shape[1] 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n[INFO] 훈련 시작... Device: {device}")
    print(f"[INFO] 입력 피처: {INPUT_SIZE}, 임베딩: {EMBEDDING_SIZE}, 모델: Residual MLP")

    base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    model = SiameseNetwork(base_net).to(device)
    criterion = CosineContrastiveLoss(margin=0.5).to(device) # Margin은 0.0~1.0 사이 권장
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    # --- [개선 1] Learning Rate Scheduler ---
    # Validation Loss가 개선되지 않으면 LR을 0.5배로 줄임
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # --- [개선 2] Early Stopping ---
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    best_model_path = os.path.join(MODEL_OUTPUT_DIR, BEST_MODEL_NAME)

    early_stopping = EarlyStopping(
        patience=PATIENCE, 
        path=best_model_path, 
        verbose=True
    )

    # === 학습 루프 ===
    for epoch in range(EPOCHS):
        model.train()
        total_train_loss = 0

        # Tqdm 진행률 표시
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for anchor, pair, label in loop:
            anchor, pair, label = anchor.to(device), pair.to(device), label.to(device)

            optimizer.zero_grad()
            out1, out2 = model(anchor, pair)
            loss = criterion(out1, out2, label)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            # 진행바에 현재 배치 손실 표시
            loop.set_postfix(loss=loss.item())

        avg_train_loss = total_train_loss / len(train_loader)

        # 검증
        avg_val_loss = validate(model, val_loader, criterion, device)

        print(f"\n[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # 스케줄러 업데이트 (Val Loss 기준)
        scheduler.step(avg_val_loss)

        # 얼리 스토핑 체크 (여기서 모델 저장도 수행됨)
        early_stopping(avg_val_loss, model)

        if early_stopping.early_stop:
            print("\n🛑 Early Stopping triggered! 학습을 조기 종료합니다.")
            break

    print("\n🎉 모든 훈련 과정 완료.")
    print(f"최종 Best Validation Loss: {early_stopping.best_loss:.4f}")

    # print("\n🎉 훈련이 완료되었습니다!")
    # print(f"가장 낮은 검증 손실(Best Val Loss): {best_val_loss:.4f}")

    # -- 7. 최종 Test Set 평가 (참고) --
    print("\n최종 Test Set으로 성능을 평가합니다...")
    # 저장된 베스트 모델 가중치를 로드
    final_base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    final_base_net.load_state_dict(torch.load(best_model_path))
    final_model = SiameseNetwork(final_base_net).to(device)

    # Test 데이터로더 생성 및 평가
    test_dataset = EventPairDataset(dataframe=test_df)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    final_test_loss = validate(final_model, test_loader, criterion, device)
    print(f"\n✅ 최종 Test Loss: {final_test_loss:.4f}")
    print("이것이 모델의 일반화 성능입니다.")

    print("\n--- [Test Set 그룹핑 결과 확인] ---")

    # 1. 필요한 라이브러리 임포트
    from sklearn.cluster import AgglomerativeClustering
    import pandas as pd
    from IPython.display import display
    
    # 2. 'final_base_net' (베스트 모델)을 평가 모드로 설정
    final_base_net.eval()
    
    # 3. Test DataFrame(test_df)에서 피처만 추출하여 텐서로 변환
    # (훈련/검증/테스트 분리 시 사용했던 key_columns를 다시 정의)
    key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
    existing_key_columns = [col for col in test_df.columns if col in key_columns]
    feature_cols = [col for col in test_df.columns if col not in existing_key_columns]
    
    test_features_np = test_df[feature_cols].values.astype(np.float32)
    test_features_tensor = torch.tensor(test_features_np).to(device)
    
    print(f"Test Set 이벤트 {len(test_features_tensor)}개에 대한 임베딩 추출 중...")
    
    # 4. 모든 Test Set 이벤트의 임베딩을 한 번에 계산
    with torch.no_grad():
        all_embeddings = final_base_net(test_features_tensor)
        all_embeddings_np = all_embeddings.cpu().numpy()
    
    print("임베딩 추출 완료.")

    # 5. Scikit-learn을 사용한 클러스터링 (그룹핑)
    # AgglomerativeClustering: 임베딩 벡터 간의 거리를 기반으로 그룹을 묶음
    
    # distance_threshold: 이 거리(임계값) 내에 있는 벡터들은 같은 클러스터로 묶임
    # **[매우 중요]** 이 임계값은 모델의 학습 결과에 따라 사용자가 직접 조정해야 합니다.
    # 보통 MARGIN(2.0) 값의 50% ~ 75% 사이에서 시작하는 것이 좋습니다.
    # clustering_threshold = MARGIN * 0.75 # (예: 2.0 * 0.75 = 1.5)

    clustering_threshold = 0.5
    
    clustering = AgglomerativeClustering(
        n_clusters=None, # 클러스터 개수를 미리 정하지 않음
        distance_threshold=clustering_threshold, # 거리 임계값 기준으로 묶음
        metric='euclidean',
        linkage='average' # 클러스터 간의 '평균' 거리를 사용
    )
    
    predicted_labels = clustering.fit_predict(all_embeddings_np)

    # 6. 원본 DataFrame에 예측된 그룹핑 결과(predicted_cluster_id)를 추가
    results_df = test_df.copy()
    results_df['predicted_cluster_id'] = predicted_labels # 모델이 예측한 그룹 ID
    
    # 7. 결과 출력
    # 'predicted_cluster_id' (예측값)으로 먼저 정렬한 뒤, 'EventTime'으로 정렬
    results_df_sorted = results_df.sort_values(by=['predicted_cluster_id', 'EventTime'])
    
    print(f"\n--- [모델 그룹핑 결과 (임계값: {clustering_threshold:.2f})] ---")
    
    # 컬럼이 너무 많으면 보기 힘드므로, 핵심 컬럼만 선택
    display_columns = ['predicted_cluster_id', 'threat_label_case_id', 'EventTime', 'threat_label_scenario']
    available_display_columns = [col for col in display_columns if col in results_df.columns]
    
    # pandas가 모든 행을 출력하도록 설정
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(results_df_sorted[available_display_columns])
        
    print("\n[결과 해석 가이드]")
    print(" - 'predicted_cluster_id'가 모델이 예측한 그룹입니다.")
    print(" - 'threat_label_case_id'가 실제 정답 그룹입니다.")
    print(" - **정답과 예측이 일치하는지** (예: predicted_cluster_id '3'에 case_id 'INCIDENT-XXX-010'만 있는지) 확인해보세요.")
    print(" - 만약 그룹이 너무 잘게 쪼개진다면 -> 'clustering_threshold' 값을 더 높여보세요 (예: 1.5 -> 1.8).")
    print(" - 만약 그룹이 너무 크게 하나로 묶인다면 -> 'clustering_threshold' 값을 더 낮춰보세요 (예: 1.5 -> 1.2).")

'final_model_input.csv' 파일 로드 및 그룹 기반 분리 시작...
데이터 분리 완료:
  - Train Set: 700개 Case ID, 1400개 이벤트
  - Val Set  : 150개 Case ID, 300개 이벤트
  - Test Set : 150개 Case ID, 300개 이벤트


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



[INFO] 훈련 시작... Device: cpu
[INFO] 입력 피처: 3136, 임베딩: 64, 모델: Residual MLP


Epoch 1/200: 100%|██████████| 22/22 [00:01<00:00, 21.79it/s, loss=1.18]



[Epoch 1] Train Loss: 3.2401 | Val Loss: 1.6534
Validation loss decreased (1.653388 --> 1.653388).  Saving model ...


...


[Epoch 28] Train Loss: 1.0143 | Val Loss: 1.1565
EarlyStopping counter: 13 out of 15


Epoch 29/200: 100%|██████████| 22/22 [00:00<00:00, 24.59it/s, loss=1.01]



[Epoch 29] Train Loss: 1.0143 | Val Loss: 1.0960
EarlyStopping counter: 14 out of 15


Epoch 30/200: 100%|██████████| 22/22 [00:00<00:00, 25.61it/s, loss=1]    



[Epoch 30] Train Loss: 1.0169 | Val Loss: 1.0395
EarlyStopping counter: 15 out of 15

🛑 Early Stopping triggered! 학습을 조기 종료합니다.

🎉 모든 훈련 과정 완료.
최종 Best Validation Loss: 1.0211

최종 Test Set으로 성능을 평가합니다...

✅ 최종 Test Loss: 1.0774
이것이 모델의 일반화 성능입니다.

--- [Test Set 그룹핑 결과 확인] ---
Test Set 이벤트 300개에 대한 임베딩 추출 중...
임베딩 추출 완료.

--- [모델 그룹핑 결과 (임계값: 0.50)] ---
      predicted_cluster_id threat_label_case_id      EventTime                              threat_label_scenario
1288                     0    AUG-CASE-E46C3E08  1761444300810  Account Manipulation - Create new Windows admi...
1196                     0    AUG-CASE-B42AF2CA  1761462060845  Account Manipulation - Create new Windows admi...
1628                     0    AUG-CASE-A40E1D46  1761486780865  Account Manipulation - Create new Windows admi...
1312                     0    AUG-CASE-8D80F337  1761544320822  Account Manipulation - Create new Windows admi...
1144                     0    AUG-CASE-80B66AC8  1761544980877  Account Ma

In [108]:
from sklearn.preprocessing import StandardScaler

In [113]:
# ==========================================
# 4. 메인 실행부 (Complete Version)
# ==========================================
if __name__ == "__main__":
    
    # --- [1] 데이터 준비 ---
    print(f"'{INPUT_FILE}' 파일 로드 및 그룹 기반 분리 시작...")
    if not os.path.exists(INPUT_FILE):
        raise FileNotFoundError(f"'{INPUT_FILE}' 파일이 없습니다.")
        
    df = pd.read_csv(INPUT_FILE)

    # 수치형 데이터 스케일링 (StandardScaler)
    # time_diff_sec 같은 큰 숫자가 모델 학습을 방해하지 않도록 0~1 수준으로 맞춤
    print("Applying StandardScaler to numerical features...")
    scaler = StandardScaler()
    
    # 스케일링할 컬럼 (현재 수치형 피처가 time_diff_sec 하나라고 가정)
    # 만약 다른 수치형 컬럼이 있다면 리스트에 추가
    numeric_cols = ['time_diff_sec'] 
    
    # 해당 컬럼이 있는지 확인 후 변환
    for col in numeric_cols:
        if col in df.columns:
            # 1. 학습 안정성을 위해 로그 변환을 먼저 적용 (시간 차이는 편차가 매우 크므로)
            # log1p는 log(x+1)을 계산하여 0인 경우에도 에러가 나지 않게 함
            df[col] = np.log1p(df[col]) 
            
            # 2. 그 후 Standard Scaling (평균 0, 분산 1) 적용
            # 주의: 차원 문제 해결을 위해 reshape(-1, 1) 사용
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
            print(f" -> '{col}' 컬럼 Log 변환 및 Scaling 완료.")
    
    unique_case_ids = df['threat_label_case_id'].unique()
    np.random.shuffle(unique_case_ids)
    
    test_size = int(len(unique_case_ids) * TEST_SPLIT_RATIO)
    val_size = int(len(unique_case_ids) * VAL_SPLIT_RATIO)
    
    test_ids = unique_case_ids[:test_size]
    val_ids = unique_case_ids[test_size : test_size + val_size]
    train_ids = unique_case_ids[test_size + val_size:]
    
    train_df = df[df['threat_label_case_id'].isin(train_ids)]
    val_df = df[df['threat_label_case_id'].isin(val_ids)]
    test_df = df[df['threat_label_case_id'].isin(test_ids)]
    
    print("데이터 분리 완료:")
    print(f"  - Train Set: {len(train_ids)}개 Case, {len(train_df)}개 이벤트")
    print(f"  - Val Set  : {len(val_ids)}개 Case, {len(val_df)}개 이벤트")
    print(f"  - Test Set : {len(test_ids)}개 Case, {len(test_df)}개 이벤트")
    
    train_dataset = EventPairDataset(dataframe=train_df)
    val_dataset = EventPairDataset(dataframe=val_df)
    test_dataset = EventPairDataset(dataframe=test_df)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # --- [2] 모델 및 학습 설정 ---
    INPUT_SIZE = train_dataset.features.shape[1] 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n[INFO] 훈련 시작... Device: {device}")
    print(f"[INFO] 입력 피처: {INPUT_SIZE}, 임베딩: {EMBEDDING_SIZE}, 모델: Residual MLP")

    base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    model = SiameseNetwork(base_net).to(device)
    # criterion = ContrastiveLoss(margin=MARGIN).to(device)
    criterion = CosineContrastiveLoss(margin=0.5).to(device) # Margin은 0.0~1.0 사이 권장
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    best_model_path = os.path.join(MODEL_OUTPUT_DIR, BEST_MODEL_NAME)
    early_stopping = EarlyStopping(patience=PATIENCE, path=best_model_path, verbose=True)

    # --- [3] 학습 루프 ---
    for epoch in range(EPOCHS):
        model.train()
        total_train_loss = 0
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for anchor, pair, label in loop:
            anchor, pair, label = anchor.to(device), pair.to(device), label.to(device)
            
            optimizer.zero_grad()
            out1, out2 = model(anchor, pair)
            loss = criterion(out1, out2, label)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, val_loader, criterion, device)
        
        print(f"\n[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        
        scheduler.step(avg_val_loss)
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("\n🛑 Early Stopping triggered!")
            break

    print("\n🎉 훈련 완료.")
    
    # --- [4] 최종 평가 및 거리 분석 (New!) ---
    print("\n--- [최종 성능 평가 및 거리 분석] ---")

    best_base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    best_base_net.load_state_dict(torch.load(best_model_path))
    best_base_net.eval()
    
    pos_dists = []
    neg_dists = []
    
    with torch.no_grad():
        for anchor, pair, label in test_loader:
            anchor, pair = anchor.to(device), pair.to(device)
            emb1 = best_base_net(anchor)
            emb2 = best_base_net(pair)
            
            # 코사인 유사도 (1: 완전일치, -1: 반대) -> 코사인 거리 (0: 완전일치, 1: 직각, 2: 반대)
            # 거리 = 1 - cosine_similarity
            cosine_sim = nn.functional.cosine_similarity(emb1, emb2)
            dists = 1 - cosine_sim
            
            dists = dists.cpu().numpy()
            labels = label.cpu().numpy().flatten()
            
            for d, l in zip(dists, labels):
                if l == 1.0:
                    pos_dists.append(d)
                else:
                    neg_dists.append(d)
    
    avg_pos = np.mean(pos_dists) if pos_dists else 0
    avg_neg = np.mean(neg_dists) if neg_dists else 0
    
    print(f"\n📊 코사인 거리 분석 결과 (0~2 범위):")
    print(f"  - 같은 사건(Positive) 평균 거리: {avg_pos:.4f} (작을수록 좋음)")
    print(f"  - 다른 사건(Negative) 평균 거리: {avg_neg:.4f} (클수록 좋음)")
    
    recommend_threshold = (avg_pos + avg_neg) / 2
    print(f"  -> 추천 Threshold: {recommend_threshold:.2f}")

    # 클러스터링 수행
    print(f"\n--- [Test Set 그룹핑 결과 (Threshold: {recommend_threshold:.2f})] ---")
    
    from sklearn.cluster import AgglomerativeClustering
    
    # 전체 데이터 임베딩 추출
    key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
    existing_key_columns = [col for col in test_df.columns if col in key_columns]
    feature_cols = [col for col in test_df.columns if col not in existing_key_columns]
    test_features_tensor = torch.tensor(test_df[feature_cols].values.astype(np.float32)).to(device)
    
    with torch.no_grad():
        all_embeddings = best_base_net(test_features_tensor).cpu().numpy()
        
    # AgglomerativeClustering은 기본적으로 Euclidean만 지원하거나 Cosine을 쓰려면 precomputed 필요
    # 간단하게 cosine distance로 변환 후 클러스터링
    from sklearn.metrics.pairwise import cosine_distances
    dist_matrix = cosine_distances(all_embeddings)
    
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=recommend_threshold, # 추천값 사용
        metric='precomputed', # 거리 행렬 직접 입력
        linkage='average'
    )
    predicted_labels = clustering.fit_predict(dist_matrix)
    
    # (이하 출력 코드는 동일)
    results_df = test_df.copy()
    results_df['predicted_cluster_id'] = predicted_labels
    
    # # 베스트 모델 로드
    # best_base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    # best_base_net.load_state_dict(torch.load(best_model_path))
    # best_base_net.eval()
    
    # # 4-1. 거리 분포 확인 (Threshold 결정을 위해 필수)
    # # Test Set에서 Positive(같은 사건)와 Negative(다른 사건) 쌍의 평균 거리를 측정합니다.
    # pos_dists = []
    # neg_dists = []
    
    # with torch.no_grad():
    #     # Test Loader를 한 바퀴 돌며 거리 측정
    #     for anchor, pair, label in test_loader:
    #         anchor, pair = anchor.to(device), pair.to(device)
    #         emb1 = best_base_net(anchor)
    #         emb2 = best_base_net(pair)
    #         dists = nn.functional.pairwise_distance(emb1, emb2).cpu().numpy()
    #         labels = label.cpu().numpy().flatten()
            
    #         for d, l in zip(dists, labels):
    #             if l == 1.0:
    #                 pos_dists.append(d)
    #             else:
    #                 neg_dists.append(d)
    
    # avg_pos = np.mean(pos_dists) if pos_dists else 0
    # avg_neg = np.mean(neg_dists) if neg_dists else 0
    
    # print(f"\n📊 거리 분석 결과:")
    # print(f"  - 같은 사건(Positive) 평균 거리: {avg_pos:.4f} (작을수록 좋음)")
    # print(f"  - 다른 사건(Negative) 평균 거리: {avg_neg:.4f} (클수록 좋음)")
    # print(f"  -> 추천 Threshold: {(avg_pos + avg_neg) / 2:.2f} (두 값의 중간)")

    # # 4-2. 클러스터링 수행 및 결과 출력
    # # 위에서 계산된 '추천 Threshold'를 사용하거나 수동 설정
    # # clustering_threshold = (avg_pos + avg_neg) / 2 
    # clustering_threshold = 0.77  # (일단 안전하게 낮은 값으로 시작)
    
    # print(f"\n--- [Test Set 그룹핑 결과 (Threshold: {clustering_threshold:.2f})] ---")
    
    # from sklearn.cluster import AgglomerativeClustering
    
    # # 임베딩 추출
    # key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
    # existing_key_columns = [col for col in test_df.columns if col in key_columns]
    # feature_cols = [col for col in test_df.columns if col not in existing_key_columns]
    # test_features_tensor = torch.tensor(test_df[feature_cols].values.astype(np.float32)).to(device)
    
    # with torch.no_grad():
    #     all_embeddings = best_base_net(test_features_tensor).cpu().numpy()
        
    # # 클러스터링
    # clustering = AgglomerativeClustering(
    #     n_clusters=None,
    #     distance_threshold=clustering_threshold,
    #     metric='euclidean',
    #     linkage='average'
    # )
    # predicted_labels = clustering.fit_predict(all_embeddings)
    
    # # 결과 저장 및 출력
    # results_df = test_df.copy()
    # results_df['predicted_cluster_id'] = predicted_labels
    results_df_sorted = results_df.sort_values(by=['predicted_cluster_id', 'EventTime'])
    
    display_columns = ['predicted_cluster_id', 'threat_label_case_id', 'EventTime', 'threat_label_scenario']
    available_display_columns = [col for col in display_columns if col in results_df.columns]
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(results_df_sorted[available_display_columns])

'final_model_input.csv' 파일 로드 및 그룹 기반 분리 시작...
Applying StandardScaler to numerical features...
 -> 'time_diff_sec' 컬럼 Log 변환 및 Scaling 완료.
데이터 분리 완료:
  - Train Set: 700개 Case, 1400개 이벤트
  - Val Set  : 150개 Case, 300개 이벤트
  - Test Set : 150개 Case, 300개 이벤트


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



[INFO] 훈련 시작... Device: cpu
[INFO] 입력 피처: 3136, 임베딩: 64, 모델: Residual MLP


Epoch 1/200:   0%|          | 0/22 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [122]:
# ==========================================
# 4. 메인 실행부 (Complete & Fixed Version)
# ==========================================
if __name__ == "__main__":
    
    # --- [1] 데이터 준비 ---
    print(f"'{INPUT_FILE}' 파일 로드 및 그룹 기반 분리 시작...")
    if not os.path.exists(INPUT_FILE):
        raise FileNotFoundError(f"'{INPUT_FILE}' 파일이 없습니다.")
        
    df = pd.read_csv(INPUT_FILE)

    # 수치형 데이터 스케일링 (StandardScaler)
    print("Applying StandardScaler to numerical features...")
    scaler = StandardScaler()
    numeric_cols = ['time_diff_sec'] 
    
    for col in numeric_cols:
        if col in df.columns:
            df[col] = np.log1p(df[col]) 
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
            print(f" -> '{col}' 컬럼 Log 변환 및 Scaling 완료.")
    
    unique_case_ids = df['threat_label_case_id'].unique()
    np.random.shuffle(unique_case_ids)
    
    test_size = int(len(unique_case_ids) * TEST_SPLIT_RATIO)
    val_size = int(len(unique_case_ids) * VAL_SPLIT_RATIO)
    
    test_ids = unique_case_ids[:test_size]
    val_ids = unique_case_ids[test_size : test_size + val_size]
    train_ids = unique_case_ids[test_size + val_size:]
    
    train_df = df[df['threat_label_case_id'].isin(train_ids)]
    val_df = df[df['threat_label_case_id'].isin(val_ids)]
    test_df = df[df['threat_label_case_id'].isin(test_ids)]
    
    print("데이터 분리 완료:")
    print(f"  - Train Set: {len(train_ids)}개 Case, {len(train_df)}개 이벤트")
    print(f"  - Val Set  : {len(val_ids)}개 Case, {len(val_df)}개 이벤트")
    print(f"  - Test Set : {len(test_ids)}개 Case, {len(test_df)}개 이벤트")
    
    train_dataset = EventPairDataset(dataframe=train_df)
    val_dataset = EventPairDataset(dataframe=val_df)
    test_dataset = EventPairDataset(dataframe=test_df)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # --- [2] 모델 및 학습 설정 ---
    INPUT_SIZE = train_dataset.features.shape[1] 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n[INFO] 훈련 시작... Device: {device}")
    print(f"[INFO] 입력 피처: {INPUT_SIZE}, 임베딩: {EMBEDDING_SIZE}, 모델: Residual MLP")

    base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    model = SiameseNetwork(base_net).to(device)
    
    # [수정됨] Cosine Loss 사용 (Margin 0.5)
    criterion = CosineContrastiveLoss(margin=0.5).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
    best_model_path = os.path.join(MODEL_OUTPUT_DIR, BEST_MODEL_NAME)
    early_stopping = EarlyStopping(patience=PATIENCE, path=best_model_path, verbose=True)

    # --- [3] 학습 루프 ---
    for epoch in range(EPOCHS):
        model.train()
        total_train_loss = 0
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for anchor, pair, label in loop:
            anchor, pair, label = anchor.to(device), pair.to(device), label.to(device)
            
            optimizer.zero_grad()
            out1, out2 = model(anchor, pair)
            loss = criterion(out1, out2, label)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, val_loader, criterion, device)
        
        print(f"\n[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        
        scheduler.step(avg_val_loss)
        early_stopping(avg_val_loss, model)
        
        if early_stopping.early_stop:
            print("\n🛑 Early Stopping triggered!")
            break

    print("\n🎉 훈련 완료.")
    
    # --- [4] 최종 평가 및 거리 분석 (Cosine 버전) ---
    print("\n--- [최종 성능 평가 및 거리 분석 (Cosine Distance)] ---")
    
    best_base_net = AdvancedMlpNetwork(INPUT_SIZE, EMBEDDING_SIZE).to(device)
    best_base_net.load_state_dict(torch.load(best_model_path))
    best_base_net.eval()
    
    pos_dists = []
    neg_dists = []
    
    with torch.no_grad():
        for anchor, pair, label in test_loader:
            anchor, pair = anchor.to(device), pair.to(device)
            emb1 = best_base_net(anchor)
            emb2 = best_base_net(pair)
            
            # 코사인 거리 = 1 - 코사인 유사도
            cosine_sim = nn.functional.cosine_similarity(emb1, emb2)
            dists = 1 - cosine_sim
            
            dists = dists.cpu().numpy()
            labels = label.cpu().numpy().flatten()
            
            for d, l in zip(dists, labels):
                if l == 1.0:
                    pos_dists.append(d)
                else:
                    neg_dists.append(d)
    
    avg_pos = np.mean(pos_dists) if pos_dists else 0
    avg_neg = np.mean(neg_dists) if neg_dists else 0
    
    print(f"\n📊 코사인 거리 분석 결과 (0~2 범위):")
    print(f"  - 같은 사건(Positive) 평균 거리: {avg_pos:.4f} (작을수록 좋음)")
    print(f"  - 다른 사건(Negative) 평균 거리: {avg_neg:.4f} (클수록 좋음)")
    
    # 추천 임계값 계산
    if avg_pos < avg_neg:
        recommend_threshold = (avg_pos + avg_neg) / 2
    else:
        print("  ⚠️ 경고: 모델이 아직 충분히 학습되지 않았습니다. (Pos >= Neg)")
        recommend_threshold = 0.3 # 기본값
        
    print(f"  -> 추천 Threshold: {recommend_threshold:.2f}")

    recommend_threshold = 0.01 # 일단 그냥 하드코딩으로 바꿈 

    # 클러스터링 수행
    print(f"\n--- [Test Set 그룹핑 결과 (Threshold: {recommend_threshold:.2f})] ---")
    
    from sklearn.cluster import AgglomerativeClustering
    from sklearn.metrics.pairwise import cosine_distances
    
    # 전체 데이터 임베딩 추출
    key_columns = ['HostName', 'EventTime', 'threat_label_case_id', 'threat_label_verdict', 'threat_label_scenario']
    existing_key_columns = [col for col in test_df.columns if col in key_columns]
    feature_cols = [col for col in test_df.columns if col not in existing_key_columns]
    test_features_tensor = torch.tensor(test_df[feature_cols].values.astype(np.float32)).to(device)
    
    with torch.no_grad():
        all_embeddings = best_base_net(test_features_tensor).cpu().numpy()
        
    # 코사인 거리 행렬 계산 및 클러스터링
    dist_matrix = cosine_distances(all_embeddings)
    
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=recommend_threshold,
        metric='precomputed',
        linkage='average'
    )
    predicted_labels = clustering.fit_predict(dist_matrix)
    
    # 결과 저장 및 출력
    results_df = test_df.copy()
    results_df['predicted_cluster_id'] = predicted_labels
    results_df_sorted = results_df.sort_values(by=['predicted_cluster_id', 'EventTime'])
    
    display_columns = ['predicted_cluster_id', 'threat_label_case_id', 'EventTime', 'threat_label_scenario']
    available_display_columns = [col for col in display_columns if col in results_df.columns]
    
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
        print(results_df_sorted[available_display_columns])

'final_model_input.csv' 파일 로드 및 그룹 기반 분리 시작...
Applying StandardScaler to numerical features...
 -> 'time_diff_sec' 컬럼 Log 변환 및 Scaling 완료.
데이터 분리 완료:
  - Train Set: 700개 Case, 1400개 이벤트
  - Val Set  : 150개 Case, 300개 이벤트
  - Test Set : 150개 Case, 300개 이벤트


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



[INFO] 훈련 시작... Device: cpu
[INFO] 입력 피처: 3136, 임베딩: 64, 모델: Residual MLP


Epoch 1/200: 100%|██████████| 22/22 [00:00<00:00, 25.72it/s, loss=0.143]



[Epoch 1] Train Loss: 0.1937 | Val Loss: 0.2355
Validation loss decreased (0.235452 --> 0.235452).  Saving model ...


...


[Epoch 34] Train Loss: 0.0209 | Val Loss: 0.1076
EarlyStopping counter: 13 out of 15


Epoch 35/200: 100%|██████████| 22/22 [00:00<00:00, 25.98it/s, loss=0.0154]



[Epoch 35] Train Loss: 0.0188 | Val Loss: 0.1116
EarlyStopping counter: 14 out of 15


Epoch 36/200: 100%|██████████| 22/22 [00:00<00:00, 25.67it/s, loss=0.0155]



[Epoch 36] Train Loss: 0.0179 | Val Loss: 0.0964
EarlyStopping counter: 15 out of 15

🛑 Early Stopping triggered!

🎉 훈련 완료.

--- [최종 성능 평가 및 거리 분석 (Cosine Distance)] ---

📊 코사인 거리 분석 결과 (0~2 범위):
  - 같은 사건(Positive) 평균 거리: 0.0326 (작을수록 좋음)
  - 다른 사건(Negative) 평균 거리: 0.7376 (클수록 좋음)
  -> 추천 Threshold: 0.39

--- [Test Set 그룹핑 결과 (Threshold: 0.01)] ---
      predicted_cluster_id threat_label_case_id      EventTime                              threat_label_scenario
956                      0    AUG-CASE-580D3FA0  1762115760849  Account Manipulation - Create new Windows admi...
957                      0    AUG-CASE-580D3FA0  1762115761009  Account Manipulation - Create new Windows admi...
364                      0    AUG-CASE-163C8C60  1762368480779  Account Manipulation - Create new Windows admi...
365                      0    AUG-CASE-163C8C60  1762368480939  Account Manipulation - Create new Windows admi...
1180                     0    AUG-CASE-B7200CB5  1762930740850  Account Manip